In [ ]:
%matplotlib inline
import os
# from osgeo import gdal
# import rasterio
# import geopandas as gpd
# import rasterio.mask
# from rasterio.windows import Window
import sys
from shapely.geometry import mapping
sys.path.append(r'E:/gitlab/geoloc2/Detecciondeterrenos')
from codigos import Generar_txt
###path de yolo dentro de computadora
os.chdir(r'C:/Users/ASUS/Inteligencia_Artificial/yolov7')
from detect_Alberto_v4 import *
from scipy.ndimage import rotate as rotate_image
from shapely import geometry

In [ ]:
%matplotlib inline
from scipy import ndimage, misc
import matplotlib.pyplot as plt

In [ ]:
os.chdir("C:/Users/ASUS/Inteligencia_Artificial/yolov7")
from detect_Alberto_v4 import modelo
Modelo=modelo(weights='Modelos/best_Fer_2.pt')

In [ ]:
import pyautogui
### se requiere una segunda pantalla o modificar los parametro
pyautogui.position() ## sobre el qgis en la esquina superior izquierda del mapa correr esta celda con ctrl+enter y modificar los parametro x1,y1

In [ ]:
from tkinter import Tk 
from PIL import ImageGrab
path_save="D:/nuevas_detecciones" ## guardar txt
path_minimagen=r"D:/imagenes"
import datetime
dim=600
x1=2289#+300
y1=159#+200
x2=x1+dim#3284
y2=y1+dim-60#703
bbox= (x1,y1,x2,y2)
import time 
le=0
vector=0
vector_final=[]
while True:
    
    k=cv2.waitKey(1) & 0xFF
    im = ImageGrab.grab(bbox,all_screens=True)
    nameimg=str(datetime.datetime.now()).replace(" ","_").replace(":","_").replace(".","_").replace("-","_")
    im=np.asarray(im)
    im=np.moveaxis(im,-1,0)
    im=np.stack([im[2],im[1],im[0]])
    im=np.moveaxis(im,0,-1)
    image_ro=im.copy()
#     image_ro=im.copy()
    cv2.imshow("1",image_ro)
    if k == ord('w'):
        image_ro1=im.copy()
        angulo=correct_orientation(im,dim=dim)[0]
    #     M = cv2.getRotationMatrix2D((dim//2,dim//2), angulo, 1)
    #     image_ro = cv2.warpAffine(image_ro, M, (dim,dim))
        image_ro1=ndimage.rotate(image_ro1,angulo,reshape=True)
        image_ro1=cv2.resize(image_ro1,(dim*2,dim*2))
        image_ro2=image_ro1.copy()

        w=image_ro1.shape[0]
        h=image_ro1.shape[1]
        
        conf=0.06
    #     vector=Modelo.detect(opt_source="cache1.png",opt_conf_thres=conf,imagen_s=image_ro)
        vector=Modelo.detect(opt_source="cache1.png",opt_conf_thres=conf,imagen_s=image_ro1)
        
    #     df_cache=vector2xy(vector,w,h,dim=dim,nameimg=nameimg)
    #     for i in range(len(df_cache)):
    #             if df_cache["Tipo"][i]=="casa":
    #                 cv2.rectangle(image_ro,df_cache["start_point_im"][i],df_cache["end_point_im"][i],(0,0,255),2)
    #             else:
    #                 cv2.rectangle(image_ro,df_cache["start_point_im"][i],df_cache["end_point_im"][i],(0,255,0),2)
        df_cache1=vector2xy(vector,w,h,dim=image_ro1.shape[0],nameimg=nameimg)
        df_cache1=df_cache1.sort_values(by="area").reset_index(drop=True)
        image_ro3=image_ro2.copy()
        image_ro4=image_ro2.copy()
        for i in range(len(df_cache1)):
            x,y=df_cache1["start_point_im"][i]
            if df_cache1["Tipo"][i]=="casa":
                cv2.rectangle(image_ro3,df_cache1["start_point_im"][i],df_cache1["end_point_im"][i],(0,0,255),2)
                cv2.putText(image_ro3, str(i), (x+50,y+50), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
            else:
                cv2.rectangle(image_ro3,df_cache1["start_point_im"][i],df_cache1["end_point_im"][i],(0,255,0),2)
                cv2.putText(image_ro3, str(i), (x+50,y+50), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
            cv2.namedWindow('finalImg1', cv2.WINDOW_NORMAL)
#             cv2.imshow("finalImg",finalImg)
            x1,y1=df_cache1.loc[i,'start_point_im']
            x2,y2=df_cache1.loc[i,'end_point_im']
            Tipo=df_cache1.loc[i,'Tipo']
            df_aux=image_ro2.copy()
            df_aux=df_aux[y1:y2,x1:x2]
            vector_final.append(list(df_cache1.loc[i,"vector_o"].split())+list(["0",nameimg]))
#             if np.sum(df_aux)>10:
#                 name=path_minimagen+"/"+Tipo+"_"+str(datetime.datetime.now()).replace(" ","_").replace(":","_").replace(".","_").replace("-","_")+".png"
#                 cv2.imwrite(name,df_aux)
        cv2.imshow('finalImg1',image_ro3)
        
        
    if k == ord('t'):
        Generar_txt([list(x) for(x) in vector_final],save_folder=path_save)
        cv2.imwrite(path_save+"/"+nameimg+".png",image_ro2)
        vector_final=[]

    if k<=27:
        break
        
cv2.destroyAllWindows()

In [ ]:
from tkinter import Tk 
from PIL import ImageGrab
path_save="D:/nuevas_detecciones" ## guardar txt
path_minimagen="D:/imagenes/nuevas"
conf=0.06
import datetime
dim=540
x1=2289#+300
y1=159#+200
x2=x1+dim#3284
y2=y1+dim#703
bbox= (x1,y1,x2,y2)
import time 
le=0
vector=0
while True:
    
    k=cv2.waitKey(1) & 0xFF
    im = ImageGrab.grab(bbox,all_screens=True)
    nameimg=str(datetime.datetime.now()).replace(" ","_").replace(":","_").replace(".","_").replace("-","_")
    im=np.asarray(im)
    im=np.moveaxis(im,-1,0)
    im=np.stack([im[2],im[1],im[0]])
    im=np.moveaxis(im,0,-1)
    image_ro=im.copy()
#     image_ro=im.copy()
    cv2.imshow("1",image_ro)
    if k == ord('w'):
        image_ro1=im.copy()
        angulo=correct_orientation(im,dim=dim)[0]
    #     M = cv2.getRotationMatrix2D((dim//2,dim//2), angulo, 1)
    #     image_ro = cv2.warpAffine(image_ro, M, (dim,dim))
        image_ro1=ndimage.rotate(image_ro1,angulo,reshape=True)
        image_ro1=cv2.resize(image_ro1,(dim*2,dim*2))
        image_ro2=image_ro1.copy()

        w=image_ro1.shape[0]
        h=image_ro1.shape[1]
        
    #     vector=Modelo.detect(opt_source="cache1.png",opt_conf_thres=conf,imagen_s=image_ro)
        vector=Modelo.detect(opt_source="cache1.png",opt_conf_thres=conf,imagen_s=image_ro1)
        
    #     df_cache=vector2xy(vector,w,h,dim=dim,nameimg=nameimg)
    #     for i in range(len(df_cache)):
    #             if df_cache["Tipo"][i]=="casa":
    #                 cv2.rectangle(image_ro,df_cache["start_point_im"][i],df_cache["end_point_im"][i],(0,0,255),2)
    #             else:
    #                 cv2.rectangle(image_ro,df_cache["start_point_im"][i],df_cache["end_point_im"][i],(0,255,0),2)
        df_cache1=vector2xy(vector,w,h,dim=image_ro1.shape[0],nameimg=nameimg)
        for i in range(len(df_cache1)):
                if df_cache1["Tipo"][i]=="casa":
                    cv2.rectangle(image_ro1,df_cache1["start_point_im"][i],df_cache1["end_point_im"][i],(0,0,255),2)
                else:
                    cv2.rectangle(image_ro1,df_cache1["start_point_im"][i],df_cache1["end_point_im"][i],(0,255,0),2)
        
        cv2.imshow("2",image_ro1)
    
    if k == ord('t'):
        Generar_txt([list(x)+list([nameimg]) for(x) in vector],save_folder=path_save)
        cv2.imwrite(path_save+"/"+nameimg+".png",image_ro2)
    if k == ord('s'):
        for cs_1 in range(len(df_cache1)):
            x1,y1=df_cache1.loc[cs_1,'start_point_im']
            x2,y2=df_cache1.loc[cs_1,'end_point_im']
            Tipo=df_cache1.loc[cs_1,'Tipo']
            df_aux=image_ro2.copy()
            df_aux=df_aux[y1:y2,x1:x2]
            if np.sum(df_aux)>10:
                name=path_minimagen+Tipo+"_"+str(datetime.datetime.now()).replace(" ","_").replace(":","_").replace(".","_").replace("-","_")+".png"
                cv2.imwrite(name,df_aux)
    if k<=27:
        break
        
cv2.destroyAllWindows()

In [ ]:
cv2.destroyAllWindows()

In [ ]:
im=np.moveaxis(im,-1,0)
np.stack([im[2],im[1],im[0]])

In [ ]:
i=3

In [ ]:
from tkinter import Tk 
from PIL import ImageGrab
import pyautogui as mo
dim=600-60
x1=2289#+300
y1=159#+200
x2=x1+dim#3284
y2=y1+dim#703
bbox= (x1,y1,x2,y2)
df_rs=pd.DataFrame()
for i in range(10):
    le=0
    vector=0
    im = ImageGrab.grab(bbox,all_screens=True)
    nameimg=str(i)
    im=np.asarray(im)
    im=np.moveaxis(im,-1,0)
    im=np.stack([im[2],im[1],im[0]])
    im=np.moveaxis(im,0,-1)

    image_ro=im.copy()
    image_ro1=im.copy()
    angulo=correct_orientation(imagen_n,dim=dim)[0]
#     M = cv2.getRotationMatrix2D((dim//2,dim//2), angulo, 1)
#     image_ro = cv2.warpAffine(image_ro, M, (dim,dim))
    image_ro1=ndimage.rotate(image_ro1,angulo,reshape=True)
#     image_ro1=cv2.resize(image_ro1,(dim,dim))
    w=image_ro1.shape[0]
    h=image_ro1.shape[1]
    conf=0.06
#     vector=Modelo.detect(opt_source="cache1.png",opt_conf_thres=conf,imagen_s=image_ro)
    vector=Modelo.detect(opt_source="cache1.png",opt_conf_thres=conf,imagen_s=image_ro1)
    df_cache=vector2xy(vector,w,h,dim=dim,nameimg=nameimg)
    df_cache["w"]=w
    df_cache["h"]=h
    df_cache["angulo"]=angulo
    df_rs=pd.concat([df_rs,df_cache],ignore_index=True)
    mo.moveTo(x1,y1)
    mo.click()
    mo.moveTo(x2,y1)
    mo.click()
    mo.moveTo(x2,y2)
    mo.click()
    mo.moveTo(x1,y2)
    mo.click()
    mo.click(button='right')
    mo.write(str(i))
    mo.keyDown("enter")
    mo.keyUp("enter")
    mo.keyDown("right")
    mo.keyUp("right")
    mo.keyDown("right")
    mo.keyUp("right")
# i+=1

In [ ]:
df_rs.reset_index(drop=True,inplace=True)
df_rs

In [ ]:
a=gpd.read_file("C:/Users/ASUS/Desktop/a.shp")
a

In [ ]:
casas=[]
terrenos=[]
for i in range(10):
    df_cache1=df_rs[df_rs["imagen"]==str(i)].reset_index(drop=True)
    shapes=mapping(a[a["id"]==i]["geometry"].values[0])
    proyecciones=shapes.get('coordinates')[0][:-1]
    angulo=df_cache1["angulo"][0]
    w=df_cache1["w"][0]
    h=df_cache1["h"][0]
    for cs_1 in range(len(df_cache1)):
        deteccion=rotacion_detect(df_cache1.loc[cs_1,'start_point_100'], df_cache1.loc[cs_1,'end_point_100'],-angulo,proyecciones,w,h,dim)
        if df_cache1.loc[cs_1,"Tipo"]=="casa":
            casas.append(deteccion)
        else:
            terrenos.append(deteccion)
        
casas

In [ ]:
gpd.GeoDataFrame(geometry=casas,crs=3857).to_file(r"E:\geoshapes/r.shp")
gpd.GeoDataFrame(geometry=terrenos,crs=3857).to_file(r"E:\geoshapes/ra.shp")

In [ ]:
df_rs[df_rs["imagen"]==str(0)]#.reset_index(drop=True)

In [ ]:
df_cache["imagen"].value_counts()

In [ ]:
mapping(a[a["id"]==1]["geometry"].values[0])

In [ ]:
df_cache1

In [ ]:
ñ

In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
img = cv.imread('cache.png')
assert img is not None, "file could not be read, check with os.path.exists()"
kernel=np.array([[0,1,2],
               [-1,1.5,1],
                [-2,-1,0]])
img=cv.filter2D(img, ddepth=-1,kernel=kernel)
# edges = cv.Canny(img,150,255)
cv.imshow("image",img)
# cv.imshow("edge",edges)
cv.waitKey()
cv.destroyAllWindows()
# plt.subplot(121),plt.imshow(img,cmap = 'gray')
# plt.title('Original Image'), plt.xticks([]), plt.yticks([])
# plt.subplot(122),plt.imshow(edges,cmap = 'gray')
# plt.title('Edge Image'), plt.xticks([]), plt.yticks([])
# plt.show()

In [ ]:
img = cv.imread('cache.png')
kernel=np.array([[0,1,2],
               [-1,1.5,1],
                [-2,-1,0]])
img=cv.filter2D(img, ddepth=-1,kernel=kernel)
a=100
r=img[:,:,0]
ret,r1 = cv.threshold(r,23+a,255,cv.THRESH_BINARY)
g=img[:,:,1]
ret,g1 = cv.threshold(g,29+a,255,cv.THRESH_BINARY)
b=img[:,:,2]
ret,b1 = cv.threshold(b,22+a,255,cv.THRESH_BINARY)

In [ ]:
ima1=np.stack([r1,g1,b1],axis=-1)
ima1 = cv.cvtColor(ima1, cv.COLOR_BGR2GRAY)
kernel=np.array([[0,0,0],
               [0,1,0],
                [0,0,0]])
ima1=cv.filter2D(ima1, ddepth=-1,kernel=kernel)
cv.imshow("a",ima1)
edges = cv.Canny(img,200,255)
# cv.imshow("image",img)
cv.imshow("edge",edges)
cv.waitKey()
cv.destroyAllWindows()

In [ ]:
f=[]
for i in r:
    for j in i:
        f.append(j)
res=pd.DataFrame(f)
res["count"]=1
res.groupby(by=0).count()
res1=res.groupby(by=0).count()
res1=res1.sort_values(by="count",ascending=False)[2:].sort_index()
res1,res1[res1["count"]==res1.max()[0]]

In [ ]:
f=[]
for i in g:
    for j in i:
        f.append(j)
res=pd.DataFrame(f)
res["count"]=1
res.groupby(by=0).count()
res1=res.groupby(by=0).count()
res1=res1.sort_values(by="count",ascending=False)[2:].sort_index()
res1,res1[res1["count"]==res1.max()[0]]

In [ ]:
f=[]
for i in b:
    for j in i:
        f.append(j)
res=pd.DataFrame(f)
res["count"]=1
res.groupby(by=0).count()
res1=res.groupby(by=0).count()
res1=res1.sort_values(by="count",ascending=False)[2:].sort_index()
res1,res1[res1["count"]==res1.max()[0]]

In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
img = cv.imread('cache.png')[:,:,0]
assert img is not None, "file could not be read, check with os.path.exists()"
plt.hist(img.ravel(),256,[0,256]); plt.show()

In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
img = cv.imread('cache.png')[:,:,1]
assert img is not None, "file could not be read, check with os.path.exists()"
plt.hist(img.ravel(),256,[0,256]); plt.show()

In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
img = cv.imread('cache.png')[:,:,2]
assert img is not None, "file could not be read, check with os.path.exists()"
plt.hist(img.ravel(),256,[0,256]); plt.show()

In [ ]:

fig = plt.figure(figsize=(10, 3))
ax1, ax2, ax3 = fig.subplots(1, 3)
img = misc.ascent()
= ndimage.rotate(img, 45, reshape=False)
full_img_45 = ndimage.rotate(img, 45, reshape=True)
ax1.imshow(img, cmap='gray')
ax1.set_axis_off()
ax2.imshow(img_45, cmap='gray')
ax2.set_axis_off()
ax3.imshow(full_img_45, cmap='gray')
ax3.set_axis_off()
fig.set_tight_layout(True)
plt.show()

In [ ]:
img_45.shape

In [ ]:
full_img_45.shape

In [ ]:
import tensorflow as tf
tf.keras.applications.MobileNet(
    input_shape=None,
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
    **kwargs
)

In [ ]:
import torchvision.models as models
alexnet = models.alexnet(pretrained=True)

In [ ]:
import torch, torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import time
from torchsummary import summary

import numpy as np
import matplotlib.pyplot as plt
import os

from PIL import Image

In [ ]:
# Applying Transforms to the Data
image_transforms = { 
    'train': transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}

In [ ]:
# Load the Data

# Set train and valid directory paths

dataset = r'C:\Users\ASUS\Inteligencia_Artificial\calsificador'

train_directory = os.path.join(dataset, 'train_pad')
valid_directory = os.path.join(dataset, 'valid')

# Batch size
bs = 32

# Number of classes
num_classes = len(os.listdir(valid_directory))  #10#2#257
print(num_classes)

# Load Data from folders
data = {
    'train': datasets.ImageFolder(root=train_directory, transform=image_transforms['train']),
    'valid': datasets.ImageFolder(root=valid_directory, transform=image_transforms['valid'])
}

# Get a mapping of the indices to the class names, in order to see the output classes of the test images.
idx_to_class = {v: k for k, v in data['train'].class_to_idx.items()}
print(idx_to_class)

# Size of Data, to be used for calculating Average Loss and Accuracy
train_data_size = len(data['train'])
valid_data_size = len(data['valid'])

# Create iterators for the Data loaded using DataLoader module
train_data_loader = DataLoader(data['train'], batch_size=bs, shuffle=True)
valid_data_loader = DataLoader(data['valid'], batch_size=bs, shuffle=True)

In [ ]:
valid_directory

In [ ]:
train_data_size, valid_data_size

In [ ]:
alexnet = models.alexnet(pretrained=True)
alexnet

In [ ]:
for param in alexnet.parameters():
    param.requires_grad = True

In [ ]:
# Change the final layer of AlexNet Model for Transfer Learning
alexnet.classifier[6] = nn.Linear(4096, num_classes)
alexnet.classifier.add_module("7", nn.LogSoftmax(dim = 1))
alexnet

In [ ]:
summary(alexnet, (3, 224, 224))

In [ ]:
# Define Optimizer and Loss Function
loss_func = nn.NLLLoss()
optimizer = optim.Adam(alexnet.parameters())
optimizer

In [ ]:
import tqdm
def train_and_validate(model, loss_criterion, optimizer, epochs=25):
    '''
    Function to train and validate
    Parameters
        :param model: Model to train and validate
        :param loss_criterion: Loss Criterion to minimize
        :param optimizer: Optimizer for computing gradients
        :param epochs: Number of epochs (default=25)
  
    Returns
        model: Trained Model with best validation accuracy
        history: (dict object): Having training loss, accuracy and validation loss, accuracy
    '''
    
    start = time.time()
    history = []
    best_acc = 0.0

    for epoch in range(epochs):
        epoch_start = time.time()
        print("Epoch: {}/{}".format(epoch+1, epochs))
        
        # Set to training mode
        model.train()
        
        # Loss and Accuracy within the epoch
        train_loss = 0.0
        train_acc = 0.0
        
        valid_loss = 0.0
        valid_acc = 0.0
        
        for i, (inputs, labels) in tqdm.tqdm(enumerate(train_data_loader),total=len(train_data_loader)):

            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Clean existing gradients
            optimizer.zero_grad()
            
            # Forward pass - compute outputs on input data using the model
            outputs = model(inputs)
            
            # Compute loss
            loss = loss_criterion(outputs, labels)
            
            # Backpropagate the gradients
            loss.backward()
            
            # Update the parameters
            optimizer.step()
            
            # Compute the total loss for the batch and add it to train_loss
            train_loss += loss.item() * inputs.size(0)
            
            # Compute the accuracy
            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))
            
            # Convert correct_counts to float and then compute the mean
            acc = torch.mean(correct_counts.type(torch.FloatTensor))
            
            # Compute total accuracy in the whole batch and add to train_acc
            train_acc += acc.item() * inputs.size(0)
            
            #print("Batch number: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}".format(i, loss.item(), acc.item()))

            
        # Validation - No gradient tracking needed
        with torch.no_grad():

            # Set to evaluation mode
            model.eval()

            # Validation loop
            for j, (inputs, labels) in tqdm.tqdm(enumerate(valid_data_loader),total=len(valid_data_loader)):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Forward pass - compute outputs on input data using the model
                outputs = model(inputs)

                # Compute loss
                loss = loss_criterion(outputs, labels)

                # Compute the total loss for the batch and add it to valid_loss
                valid_loss += loss.item() * inputs.size(0)

                # Calculate validation accuracy
                ret, predictions = torch.max(outputs.data, 1)
                correct_counts = predictions.eq(labels.data.view_as(predictions))

                # Convert correct_counts to float and then compute the mean
                acc = torch.mean(correct_counts.type(torch.FloatTensor))

                # Compute total accuracy in the whole batch and add to valid_acc
                valid_acc += acc.item() * inputs.size(0)

                #print("Validation Batch number: {:03d}, Validation: Loss: {:.4f}, Accuracy: {:.4f}".format(j, loss.item(), acc.item()))
            
        # Find average training loss and training accuracy
        avg_train_loss = train_loss/train_data_size 
        avg_train_acc = train_acc/train_data_size

        # Find average training loss and training accuracy
        avg_valid_loss = valid_loss/valid_data_size 
        avg_valid_acc = valid_acc/valid_data_size

        history.append([avg_train_loss, avg_valid_loss, avg_train_acc, avg_valid_acc])
                
        epoch_end = time.time()
    
        print("Epoch : {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, \n\t\tValidation : Loss : {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(epoch+1, avg_train_loss, avg_train_acc*100, avg_valid_loss, avg_valid_acc*100, epoch_end-epoch_start))
        
        # Save if the model has best accuracy till now
        #torch.save(model, dataset+'_model_'+str(epoch)+'.pt')
            
    return model, history

In [ ]:
device = torch.device(0)
device

In [ ]:
# device = torch.device(0 if torch.cuda.is_available() else "cpu")
device = torch.device(0)
num_epochs = 2
trained_model, history = train_and_validate(alexnet, loss_func, optimizer, num_epochs)

torch.save(history, dataset+'_history.pt')

In [ ]:
history = np.array(history)
plt.plot(history[:,0:2])
plt.legend(['Tr Loss', 'Val Loss'])
plt.xlabel('Epoch Number')
plt.ylabel('Loss')
plt.ylim(0,10)
plt.savefig(dataset+'_loss_curve.png')
plt.show()

In [ ]:
plt.plot(history[:,2:4])
plt.legend(['Tr Accuracy', 'Val Accuracy'])
plt.xlabel('Epoch Number')
plt.ylabel('Accuracy')
plt.ylim(0,1)
plt.savefig(dataset+'_accuracy_curve.png')
plt.show()

In [ ]:
def predict(model, test_image_name):
    '''
    Function to predict the class of a single test image
    Parameters
        :param model: Model to test
        :param test_image_name: Test image

    '''
    
    transform = image_transforms['test']

    test_image = Image.open(test_image_name)
    plt.imshow(test_image)
    
    test_image_tensor = transform(test_image)

#     if torch.cuda.is_available():
#         test_image_tensor = test_image_tensor.view(1, 3, 224, 224).cuda()
#     else:
    test_image_tensor = test_image_tensor.view(1, 3, 224, 224)
    
    with torch.no_grad():
        model.eval()
        # Model outputs log probabilities
        out = model(test_image_tensor)
        ps = torch.exp(out)
        topk, topclass = ps.topk(3, dim=1)
        for i in range(3):
            print("Predcition", i+1, ":", idx_to_class[topclass.numpy()[0][i]], ", Score: ", topk.numpy()[0][i])

In [ ]:
predict(trained_model, r"C:\Users\ASUS\Inteligencia_Artificial\calsificador\valid\casas\814_25.png")

In [ ]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
img = cv.imread(r"C:\Users\ASUS\Inteligencia_Artificial\yolov7\cache.png", cv.IMREAD_GRAYSCALE)
assert img is not None, "file could not be read, check with os.path.exists()"
img = cv.medianBlur(img,5)
ret,th1 = cv.threshold(img,127,255,cv.THRESH_BINARY)
th2 = cv.adaptiveThreshold(img,255,cv.ADAPTIVE_THRESH_MEAN_C,\
            cv.THRESH_BINARY,11,2)
th3 = cv.adaptiveThreshold(img,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv.THRESH_BINARY,11,2)
import cv2
cv2.imshow("th1",th1)
cv2.imshow("th2",th2)
cv2.imshow("th3",th3)
cv2.waitKey()
cv2.destroyAllWindows()
# titles = ['Original Image', 'Global Thresholding (v = 127)',
#             'Adaptive Mean Thresholding', 'Adaptive Gaussian Thresholding']
# # images = [img, th1, th2, th3]
# # for i in range(4):
# #     plt.subplot(2,2,i+1),plt.imshow(images[i],'gray')
# #     plt.title(titles[i])
# #     plt.xticks([]),plt.yticks([])
# # plt.show()

In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
img = cv.imread(r"C:\Users\ASUS\Inteligencia_Artificial\yolov7\cache1.png")
assert img is not None, "file could not be read, check with os.path.exists()"
color = ('b','g','r')
for i,col in enumerate(color):
    histr = cv.calcHist([img],[i],None,[256],[0,256])
    plt.plot(histr,color = col)
    plt.xlim([0,256])
plt.show()